In [8]:
import nltk
import numpy as np
import pandas as pd
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx


In [10]:
text= input("Enter text")


Enter textEnter textLorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum vulputate varius elit, sed dapibus est interdum sit amet. Vestibulum mi tellus, ornare quis fringilla vitae, interdum non libero. Vivamus non eros nec lacus fermentum consequat sit amet vitae nisi. In lectus dolor, dignissim eu gravida at, placerat vitae ipsum. In viverra a erat non interdum. Fusce sollicitudin ultricies interdum. Vivamus mollis dolor risus, ut consectetur quam vehicula consectetur. Suspendisse faucibus erat eu urna interdum mollis. Aliquam commodo et erat eu faucibus.  Donec iaculis ipsum et diam iaculis vestibulum. Mauris a nibh eget mauris pharetra varius vitae quis dui. Morbi eu erat consequat, commodo nisl vitae, pharetra risus. Sed congue faucibus mauris vel lobortis. In hac habitasse platea dictumst. Aliquam venenatis aliquam arcu, eu sollicitudin nulla imperdiet vitae. Phasellus consequat sagittis vestibulum.  Donec fermentum felis ipsum, id fringilla metus commodo at. Vivam

In [11]:
sentences=sent_tokenize(text)

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
!pip install textstat

In [14]:
import textstat
print("Num sentences:", textstat.sentence_count(text))
print("Num words:", len(text.split()))

Num sentences: 57
Num words: 443


In [15]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

In [16]:
w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)

sentence_embeddings=[[w2v.wv[word][0] for word in words] for words in sentence_tokens]

max_len=max([len(tokens) for tokens in sentence_tokens])

sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])

for i,row_embedding in enumerate(sentence_embeddings):
    
    for j,column_embedding in enumerate(sentence_embeddings):
        
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)

In [17]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)


In [18]:
summary=int(input("enter the number of sentences you want to display:"))

top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}

top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:summary])

enter the number of sentences you want to display:20


In [19]:
for sent in sentences:
    
    if sent in top.keys():
        
        print(sent)

Enter textLorem ipsum dolor sit amet, consectetur adipiscing elit.
In lectus dolor, dignissim eu gravida at, placerat vitae ipsum.
Suspendisse faucibus erat eu urna interdum mollis.
Aliquam commodo et erat eu faucibus.
Donec iaculis ipsum et diam iaculis vestibulum.
Sed congue faucibus mauris vel lobortis.
Vivamus metus dui, tristique vitae gravida at, sollicitudin a libero.
Nullam non nunc sagittis, dictum nibh at, tincidunt massa.
Mauris in sapien cursus, porttitor urna vel, semper metus.
Sed mollis lorem enim, ut facilisis eros tempor suscipit.
Sed in sapien elementum eros posuere sagittis.
Nam a pulvinar odio, sed imperdiet nibh.
Nunc eleifend ante at nisl commodo, at faucibus mauris porta.
Suspendisse feugiat sagittis elit, a consectetur neque imperdiet sit amet.
Vivamus consequat eleifend orci vel pretium.
Pellentesque interdum nulla id odio rutrum gravida.
Etiam facilisis accumsan turpis sit amet mollis.
Aliquam fringilla vel quam vitae ornare.
Nullam dignissim sodales lorem, eu